## 8. Robustness and Performance

### 66 Consider `contextlib` and `with` Statements for Reusable `try`/`finally` Behavior

In [1]:
from threading import Lock

In [2]:
lock = Lock()
with lock:
    # Do something while maintaining an invariant
    pass

In [3]:
lock.acquire()
try:
    # Do something while maintaining an invariant
    pass
finally:
    lock.release()

In [4]:
import logging

In [5]:
#logging.getLogger().setLevel(logging.WARNING)

In [6]:
assert logging.getLogger().getEffectiveLevel() == logging.WARNING

In [7]:
def my_function():
    logging.debug('Some debug data')
    logging.error('Error log here')
    logging.debug('More debug data')

In [8]:
my_function()

ERROR:root:Error log here


In [9]:
from contextlib import contextmanager

In [10]:
@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield
    finally:
        logger.setLevel(old_level)

In [11]:
with debug_logging(logging.DEBUG):
    print('* Inside:')
    my_function()

DEBUG:root:Some debug data
ERROR:root:Error log here
DEBUG:root:More debug data


* Inside:


In [12]:
print('* After:')
my_function()

ERROR:root:Error log here


* After:


In [13]:
with open('my_output.txt', 'w') as handle:
    handle.write('This is some data!')

In [14]:
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)

In [15]:
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug(f'This is a message for {logger.name}!')
    logging.debug('This will not print')

DEBUG:my-log:This is a message for my-log!


In [16]:
logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

ERROR:my-log:Error will print


In [17]:
with log_level(logging.DEBUG, 'other-log') as logger:
    logger.debug(f'This is a message for {logger.name}!')
    logging.debug('This will not print')

DEBUG:other-log:This is a message for other-log!


> - `with` 문을 사용하면 `try`/`finally` 블록을 통해 사용해야 하는 로직을 재활용하면서 시각적인 잡음도 줄일 수 있다.
> - `contextlib` 내장 모듈이 제공하는 `contextmanager` 데코레이터를 사용하면 여러분이 만든 함수를 `with` 문에 사용할 수 있다.
> - 컨텍스트 매니저가 `yield`하는 값은 `with` 문의 `as` 부분에 전달된다. 이를 활용하면 특별한 컨텍스트 내부에서 실행되는 코드 안에서 직접 그 컨텍스트에 접근할 수 있다.